# Import Dependencies

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# Video feed
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    cv2.imshow('Mediapipe Feed',frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 1. Making Detections

In [3]:
cap = cv2.VideoCapture(0)

# setting up media pipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False                     # do let anyone edit the image of webcam
        
        # make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True                      # to draw on image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections using mp_drawing module
        mp_drawing.draw_landmarks(  image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [23]:
# gives us coordinates for the nodes
# results.pose_landmarks       

# tells us about the connections between nodes
# mp_pose.POSE_CONNECTIONS

# to understand the drawing function agruments
# mp_drawing.draw_landmarks??

# 2.  Determining Joints

![title](Images\Ladmarks.png)

In [25]:
cap = cv2.VideoCapture(0)

# setting up media pipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False                     # do let anyone edit the image of webcam
        
        # make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True                      # to draw on image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extracting Landmarks
        
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass                # if we have no detectiong we pass
        
        
        # Render detections using mp_drawing module
        mp_drawing.draw_landmarks(  image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [37]:
len(landmarks)

33

In [30]:
# Checking all the names of landmarks
for lnd in mp_pose.PoseLandmark:
    print(lnd)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [35]:
# to find a coordiantes of a particular joint
print(landmarks[mp_pose.PoseLandmark.LEFT_EYE].x)

0.6868488788604736


# Calculate Angles

In [55]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    
    b_c_diff = np.arctan2(c[1]-b[1], c[0]-b[0])       # elbow_wrist_diff
    a_b_diff = np.arctan2(a[1]-b[1], a[0]-b[0])       # shoulder elbow_diff             
    
    # documnted way to find radian and angle from joints
    radians = b_c_diff- a_b_diff
    angle = np.abs(radians*180/np.pi)
    
    # coz the wrrist wont move more than 180* 
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

In [56]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]

In [57]:
print(shoulder, elbow, wrist)
# shows the angle
calculate_angle(shoulder, elbow, wrist)

[0.9267154932022095, 0.9531013369560242] [1.0829576253890991, 1.3321505784988403] [1.0598219633102417, 1.3321505784988403]


67.59879724639922

In [62]:
def find_web_cam_dim():
    cap = cv2.VideoCapture(0)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    return (width, height)

In [64]:
list(find_web_cam_dim())

[640.0, 480.0]

In [65]:
# as u can see the coordinates of shoulder,wrist and elbow are given in normalised form by mediapipe
# so to find the coordinate of the elbow in python web camera we multiply web cam dimensions to normalised value
# hence corrdinate of elbow will be
tuple(np.multiply(elbow,list(find_web_cam_dim())).astype(int))


(693, 639)

In [74]:
cap = cv2.VideoCapture(0)

# setting up media pipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False                     # do let anyone edit the image of webcam
        
        # make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True                      # to draw on image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extracting Landmarks
        
        try:
            landmarks = results.pose_landmarks.landmark
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
            # calculating angle
            angle = calculate_angle(shoulder, elbow, wrist)
#             print(angle)
            # visualise angle
            cv2.putText(image, str(angle), 
                    tuple(np.multiply(elbow,[640,480]).astype(int)),       # explained above
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0, 255), 2, cv2.LINE_AA            # font,opacity,color,width
            )
        
        except:
            pass                # if we have no detectiong we pass
        
        
        # Render detections using mp_drawing module
        mp_drawing.draw_landmarks(  image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 4. Curl Counter

# we use two variables - 
Stage  - for saving the state for of up and down <br>
counter - increasing reps number at every up state

In [75]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            
            if angle > 160:
                stage = "down"
            # if stage down but angle shows up/low we update it 
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1

                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

